<a href="https://colab.research.google.com/github/nicolemichaud03/Recipe-Recommender-System/blob/main/Colab_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Capstone Project - Recipe/Meal Recommendation System

In [11]:
!pip install scikit-surprise
# !conda install -y -c conda-forge scikit-surprise # If you use conda on a non-Colab environment



In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
import numpy as np
import pickle
from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.stem import WordNetLemmatizer
from surprise.model_selection import train_test_split
import io
from surprise import SVD, BaselineOnly, SVDpp, NMF
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
user_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/capstone_data/RAW_interactions.csv')

In [4]:
recipe_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/capstone_data/RAW_recipes.csv')

In [5]:
user_data.head()


,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [6]:
#recipe_data = pd.read_csv('data/RAW_recipes.csv')
recipe_data.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [7]:
recipe_df = recipe_data.drop(columns=['contributor_id', 'submitted', 'tags', 'nutrition', 'description', 'ingredients', 'n_steps', 'steps', 'n_ingredients'])
recipe_df.head()

,name,id,minutes
0,arriba baked winter squash mexican style,137739,55
1,a bit different breakfast pizza,31490,30
2,all in the kitchen chili,112140,130
3,alouette potatoes,59389,45
4,amish tomato ketchup for canning,44061,190


In [8]:
from surprise import Reader, Dataset
# read in values as Surprise dataset

reader = Reader()
recipe_df = Dataset.load_from_df(recipe_df, reader)


In [9]:
print(recipe_df)

In [10]:
#recipes_PP = pd.read_csv('data/PP_recipes.csv')
#recipes_PP.head()

In [11]:
#users_PP = pd.read_csv('data/PP_users.csv')
#users_PP.head()

In [12]:
#users_PP.info()

In [13]:
#users_PP['ratings'] = users_PP['ratings'].astype(float)
user_df = user_data.drop(columns=['date', 'review'])
#users_PP = users_PP.rename(columns = {'u': 'user', 'items':'recipes'})
user_df.head()

,user_id,recipe_id,rating
0,38094,40893,4
1,1293707,40893,5
2,8937,44394,4
3,126440,85009,5
4,57222,85009,5


In [14]:
user_df = Dataset.load_from_df(user_df, reader)
user_df

In [15]:
#pickled_map = pd.read_pickle('data/ingr_map.pkl')
#pickled_map

## Data Exploration

In [29]:
user_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1132367 entries, 0 to 1132366
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   user_id    1132367 non-null  int64 
 1   recipe_id  1132367 non-null  int64 
 2   date       1132367 non-null  object
 3   rating     1132367 non-null  int64 
 4   review     1132198 non-null  object
dtypes: int64(3), object(2)
memory usage: 43.2+ MB


In [30]:
print(len(user_data['user_id'].unique()))

226570


There are 226,570 unique users in this dataset

In [31]:
recipe_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   name            231636 non-null  object
 1   id              231637 non-null  int64 
 2   minutes         231637 non-null  int64 
 3   contributor_id  231637 non-null  int64 
 4   submitted       231637 non-null  object
 5   tags            231637 non-null  object
 6   nutrition       231637 non-null  object
 7   n_steps         231637 non-null  int64 
 8   steps           231637 non-null  object
 9   description     226658 non-null  object
 10  ingredients     231637 non-null  object
 11  n_ingredients   231637 non-null  int64 
dtypes: int64(5), object(7)
memory usage: 21.2+ MB


In [32]:
recipe_data = recipe_data.drop(columns=['contributor_id', 'submitted', 'nutrition', 'steps'])

In [33]:
print(len(recipe_data['id'].unique()))

231637


There are 231,637 unique recipes in this dataset.

With 226,570 users and 231,637 recipes, there are less users than there are recipes. Therefore, it is probably best for our recommender system to be user-user based.

## Preprocessing

The text data in description needs to be cleaned to ensure all punctuation is removed and words are all lowercase. The text data in the other columns looks pretty well cleaned already, but we still need to remove stopwords.

In [34]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [34]:
# Creating a function to perform cleaning steps at once
stopwords_list = stopwords.words('english')

no_bad_chars = re.compile('[!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n - ]')
no_nums = re.compile('[\d-]')

def clean_text(text):
    #text = no_nums.sub('', text)
    text = no_bad_chars.sub(' ', text)
    text = text.lower()
    text = ' '.join(word for word in text.split() if word not in stopwords_list)
    return text


In [35]:
#recipe_data['tags'] = recipe_data['tags'].astype(str)
tags_cleaned = recipe_data['tags'].apply(clean_text)
tags_cleaned

0         '60 minutes less' 'time make' 'course' 'main i...
1         '30 minutes less' 'time make' 'course' 'main i...
2         'time make' 'course' 'preparation' 'main dish'...
3         '60 minutes less' 'time make' 'course' 'main i...
4         'weeknight' 'time make' 'course' 'main ingredi...
                                ...                        
231632    'ham' '60 minutes less' 'time make' 'course' '...
231633    '15 minutes less' 'time make' 'course' 'prepar...
231634    '60 minutes less' 'time make' 'course' 'main i...
231635    '30 minutes less' 'time make' 'course' 'prepar...
231636    '30 minutes less' 'time make' 'course' 'prepar...
Name: tags, Length: 231637, dtype: object

In [36]:
GF = []
for row in recipe_data['tags']:
    if "gluten-free" in row : GF.append(1)
    elif "gluten free" in row : GF.append(1)
    else: GF.append(0)

In [37]:
recipe_data['GF'] = GF

In [38]:
recipe_data['GF'].value_counts()

0    225894
1      5743
Name: GF, dtype: int64

In [39]:
#Ingredient lists for diet filtering:
vegan = ['ham', 'beef', 'meat', 'chicken', 'pork', 'bacon', 'sausage', 'lamb', 'veal', 'turkey', 'steak', 'rib', 'frankfurter', 'duck', 'poultry', 'goat', 'liver', 'hen', 'quail', 'brisket', 'goose','fish', 'shrimp', 'seafood', 'crab', 'lobster', 'clam', 'oyster', 'scallop', 'mussel', 'cod', 'salmon', 'halibut', 'shellfish', 'roe', 'tuna', 'caviar', 'pollock', 'yellowtail', 'squid', 'calamari', 'octopus', 'crawfish', 'crayfish', 'sardine', 'trout', 'flounder', 'anchovy', 'bass', 'haddock', 'sole','egg', 'honey','milk', 'cheese', 'yogurt', 'mayonnaise', 'butter', 'margarine', 'cream']

vegetarian = ['ham', 'beef', 'meat', 'chicken', 'pork', 'bacon', 'sausage', 'lamb', 'veal', 'turkey', 'steak', 'rib', 'frankfurter', 'duck', 'poultry', 'goat', 'liver', 'hen', 'quail', 'brisket', 'goose','fish', 'shrimp', 'seafood', 'crab', 'lobster', 'clam', 'oyster', 'scallop', 'mussel', 'cod', 'salmon', 'halibut', 'shellfish', 'roe', 'tuna', 'caviar', 'pollock', 'yellowtail', 'squid', 'calamari', 'octopus', 'crawfish', 'crayfish', 'sardine', 'trout', 'flounder', 'anchovy', 'bass', 'haddock', 'sole']

In [40]:
basic_token_pattern = r"(?u)\b\w\w+\b"
tokenizer = RegexpTokenizer(basic_token_pattern)

In [41]:
recipe_data['vegetarian'] = None
recipe_data['vegan'] = None



In [42]:
vege_pattern = '|'.join(vegetarian)
vegan_pattern = '|'.join(vegan)


recipe_data.vegetarian = recipe_data.ingredients.str.contains(vege_pattern)
recipe_data.vegan = recipe_data.ingredients.str.contains(vegan_pattern)

In [43]:
recipe_data.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,GF,vegetarian,vegan
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,0,False,True
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,0,True,True
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,0,True,True
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,0,False,True
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,0,False,False


In [44]:
new_vals = {'False': 1, 'True': 0}

In [45]:
recipe_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   name            231636 non-null  object
 1   id              231637 non-null  int64 
 2   minutes         231637 non-null  int64 
 3   contributor_id  231637 non-null  int64 
 4   submitted       231637 non-null  object
 5   tags            231637 non-null  object
 6   nutrition       231637 non-null  object
 7   n_steps         231637 non-null  int64 
 8   steps           231637 non-null  object
 9   description     226658 non-null  object
 10  ingredients     231637 non-null  object
 11  n_ingredients   231637 non-null  int64 
 12  GF              231637 non-null  int64 
 13  vegetarian      231637 non-null  bool  
 14  vegan           231637 non-null  bool  
dtypes: bool(2), int64(6), object(7)
memory usage: 23.4+ MB


In [46]:
recipe_data['vegetarian'] = recipe_data['vegetarian'].astype(str)
recipe_data['vegetarian'] = recipe_data['vegetarian'].replace({'False': 1, 'True': 0})

In [47]:
recipe_data['vegan'] = recipe_data['vegan'].astype(str)
recipe_data['vegan'] = recipe_data['vegan'].replace({'False': 1, 'True': 0})

In [48]:
recipe_data.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,GF,vegetarian,vegan
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,0,1,0
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,0,0,0
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,0,0,0
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,0,1,0
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,0,1,1


In [49]:
recipe_data['vegetarian'].value_counts()

1    134596
0     97041
Name: vegetarian, dtype: int64

In [50]:
recipe_data['vegan'].value_counts()

0    197961
1     33676
Name: vegan, dtype: int64

In [51]:
#Making column names match and merging dfs to classify user diets based on the recipes they've used
recipe_data = recipe_data.rename(columns = {'id': 'recipe_id'})

user_diets = pd.merge(user_data, recipe_data, on='recipe_id', how='left')



In [52]:
user_ratings = user_diets.copy()
user_ratings = user_ratings.drop(columns=['date', 'review', 'name', 'minutes', 'tags', 'n_steps', 'description', 'ingredients', 'n_ingredients'])
user_ratings.head()

KeyError: "['date', 'review'] not found in axis"

In [ ]:
user_diets = user_diets.drop(columns=['date', 'review', 'rating', 'name', 'minutes', 'tags', 'n_steps', 'description', 'ingredients', 'n_ingredients'])
user_diets.head()

In [ ]:
user_diets_count = user_diets.groupby(['user_id'])['recipe_id'].count()
user_diets_count.head()

In [53]:
user_diets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1132367 entries, 0 to 1132366
Data columns (total 17 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   user_id         1132367 non-null  int64 
 1   recipe_id       1132367 non-null  int64 
 2   rating          1132367 non-null  int64 
 3   name            1132366 non-null  object
 4   minutes         1132367 non-null  int64 
 5   contributor_id  1132367 non-null  int64 
 6   submitted       1132367 non-null  object
 7   tags            1132367 non-null  object
 8   nutrition       1132367 non-null  object
 9   n_steps         1132367 non-null  int64 
 10  steps           1132367 non-null  object
 11  description     1108857 non-null  object
 12  ingredients     1132367 non-null  object
 13  n_ingredients   1132367 non-null  int64 
 14  GF              1132367 non-null  int64 
 15  vegetarian      1132367 non-null  int64 
 16  vegan           1132367 non-null  int64 
dtypes: int64

In [54]:
len(user_diets['user_id'].unique())

226570

In [55]:
user_diets = (user_diets.groupby(['user_id']).agg({'recipe_id': lambda x: x.tolist(), 'GF': sum, 'vegetarian':sum, 'vegan': sum}).reset_index())
user_diets.head()

,user_id,recipe_id,GF,vegetarian,vegan
0,1533,"[116345, 32907, 14750, 24136, 63598, 83375, 35...",11,57,8
1,1535,"[349022, 50022, 78834, 47474, 230720, 14111, 8...",20,630,114
2,1581,[341050],0,0,0
3,1634,"[16512, 158215, 34533, 44459, 26212, 168194, 3...",3,35,9
4,1676,"[34233, 99156, 66799, 166273, 250600, 179836, ...",3,4,2


In [56]:
user_diets['is_vegetarian'] = None
user_diets['is_vegan'] = None
user_diets['is_GF'] = None
user_diets.head()

,user_id,recipe_id,GF,vegetarian,vegan,is_vegetarian,is_vegan,is_GF
0,1533,"[116345, 32907, 14750, 24136, 63598, 83375, 35...",11,57,8,None,None,None
1,1535,"[349022, 50022, 78834, 47474, 230720, 14111, 8...",20,630,114,None,None,None
2,1581,[341050],0,0,0,None,None,None
3,1634,"[16512, 158215, 34533, 44459, 26212, 168194, 3...",3,35,9,None,None,None
4,1676,"[34233, 99156, 66799, 166273, 250600, 179836, ...",3,4,2,None,None,None


In [57]:
count = 0
for str_list in user_diets['recipe_id']:
    for name in str_list:
        if name == "arts":
            count += 1

print(count)

0


In [58]:
user_diets['recipe_totals'] = user_diets['recipe_id'].str.len()

In [59]:
user_diets.head()

,user_id,recipe_id,GF,vegetarian,vegan,is_vegetarian,is_vegan,is_GF,recipe_totals
0,1533,"[116345, 32907, 14750, 24136, 63598, 83375, 35...",11,57,8,None,None,None,128
1,1535,"[349022, 50022, 78834, 47474, 230720, 14111, 8...",20,630,114,None,None,None,794
2,1581,[341050],0,0,0,None,None,None,1
3,1634,"[16512, 158215, 34533, 44459, 26212, 168194, 3...",3,35,9,None,None,None,60
4,1676,"[34233, 99156, 66799, 166273, 250600, 179836, ...",3,4,2,None,None,None,31


In [60]:
user_diets['is_vegetarian'] = np.where(user_diets['vegetarian'] >= ((user_diets['recipe_totals'])*(.75)), True, False)
user_diets.head()

,user_id,recipe_id,GF,vegetarian,vegan,is_vegetarian,is_vegan,is_GF,recipe_totals
0,1533,"[116345, 32907, 14750, 24136, 63598, 83375, 35...",11,57,8,False,None,None,128
1,1535,"[349022, 50022, 78834, 47474, 230720, 14111, 8...",20,630,114,True,None,None,794
2,1581,[341050],0,0,0,False,None,None,1
3,1634,"[16512, 158215, 34533, 44459, 26212, 168194, 3...",3,35,9,False,None,None,60
4,1676,"[34233, 99156, 66799, 166273, 250600, 179836, ...",3,4,2,False,None,None,31


In [61]:
user_diets['is_vegetarian'].value_counts()

True     114857
False    111713
Name: is_vegetarian, dtype: int64

In [62]:
user_diets['is_vegan'] = np.where(user_diets['vegan'] >= ((user_diets['recipe_totals'])*(.75)), True, False)
user_diets['is_vegan'].value_counts()

False    206565
True      20005
Name: is_vegan, dtype: int64

In [63]:
user_diets['is_GF'] = np.where(user_diets['GF'] >= ((user_diets['recipe_totals'])*(.75)), True, False)
user_diets['is_GF'].value_counts()

False    220978
True       5592
Name: is_GF, dtype: int64

In [64]:
user_diets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226570 entries, 0 to 226569
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   user_id        226570 non-null  int64 
 1   recipe_id      226570 non-null  object
 2   GF             226570 non-null  int64 
 3   vegetarian     226570 non-null  int64 
 4   vegan          226570 non-null  int64 
 5   is_vegetarian  226570 non-null  bool  
 6   is_vegan       226570 non-null  bool  
 7   is_GF          226570 non-null  bool  
 8   recipe_totals  226570 non-null  int64 
dtypes: bool(3), int64(5), object(1)
memory usage: 11.0+ MB


## Modeling

Seeing what model works best for this data:

In [68]:
benchmark = []
# Iterate over all algorithms

algorithms = [SVD(), SVDpp(), NMF(), BaselineOnly()]

print ("Attempting: ", str(algorithms), '\n\n\n')

for algorithm in algorithms:
    print("Starting: " ,str(algorithm))
    # Perform cross validation
    results = cross_validate(algorithm, user_df, measures=['RMSE'], cv=3, verbose=False)
    # results = cross_validate(algorithm, data, measures=['RMSE','MAE'], cv=3, verbose=False)

    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    print("Done: " ,str(algorithm), "\n\n")

print ('\n\tDONE\n')

Attempting:  [<surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7c9330ac2ec0>, <surprise.prediction_algorithms.matrix_factorization.SVDpp object at 0x7c9330ac3160>, <surprise.prediction_algorithms.matrix_factorization.NMF object at 0x7c9330ac3640>, <surprise.prediction_algorithms.baseline_only.BaselineOnly object at 0x7c9330ac3010>] 



Starting:  <surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7c9330ac2ec0>


<ipython-input-68-e081e8cc150a>:16: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))


Done:  <surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7c9330ac2ec0> 


Starting:  <surprise.prediction_algorithms.matrix_factorization.SVDpp object at 0x7c9330ac3160>


KeyboardInterrupt: 

In [ ]:
surprise_results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')

In [ ]:
surprise_results

,test_rmse,fit_time,test_time
Algorithm,,,
BaselineOnly,1.215818,9.472317,3.827044
SVD,1.221989,27.853095,4.902354
SVDpp,1.226767,1445.643159,182.459326
NMF,1.360473,77.439752,5.232391


It looks like the baseline only model performed best and took the least amount of time. SVD was second best. Can we improve either of these with parameter tuning?

In [ ]:
results_df = pd.DataFrame.from_dict(BaselineOnly.cv_results)

In [69]:
params1 = {'n_factors': [10, 20, 50],
           'reg_all': [0.01, 0.02, 0.05],
           'lr_all': [0.002]
           }
g_s_bo = GridSearchCV(BaselineOnly, param_grid=params1, n_jobs=-1)
g_s_bo.fit(user_df)



                    # 'bsl_options': {'method': ['als', 'sgd'],
        #'reg': [1, 2]}

TypeError: BaselineOnly.__init__() got an unexpected keyword argument 'n_factors'

In [ ]:
# print out optimal parameters for SVD after GridSearch
print(g_s_bo.best_score)
print(g_s_bo.best_params)

In [70]:
params = {'n_factors': [10,20,50],
         'reg_all': [ 0.02, 0.05, 0.07]}
g_s_svd = GridSearchCV(SVD, param_grid=params, n_jobs=-1)
g_s_svd.fit(user_df)

KeyboardInterrupt: 

In [ ]:
# print out optimal parameters for SVD after GridSearch
print(g_s_svd.best_score)
print(g_s_svd.best_params)

Best parameters for SVD were n_factors=10 and reg_all=0.05.

In [65]:
#train-test split the data
trainset, testset = train_test_split(user_df, test_size=0.25)

____

In [112]:
import collections
#from surprise documentation: https://surprise.readthedocs.io/en/stable/FAQ.html
skips = []
def get_top_n(predictions, n=3):


    # First map the predictions to each user.
    top_n = defaultdict(list)
    for user_id, iid, true_r, est, _ in predictions:
        top_n[user_id].append((iid, est))



    # skip ones that aren't for the right diet type
    for prediction in predictions:
      for user in user_diets:
        if user_diets['is_vegetarian'] == "True":
          skips.append(prediction)
          #= predictions[prediction-1].drop(prediction-1)
        elif user_diets['is_vegan'] == "True":
          skips.append(prediction)
            #filtered_preds = predictions[prediction-1].drop(prediction-1)
        elif user_diets['is_GF'] == "True":
            skips.append(prediction)
              #filtered_preds = predictions[prediction-1].drop(prediction-1)



    # Then sort the predictions for each user and retrieve the k highest ones.
    for user_id, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[user_id] = user_ratings[:n] - skips

    return top_n



In [108]:
user_data = user_data.drop(columns= ['date', 'review'])

KeyError: "['date', 'review'] not found in axis"

In [109]:

# First train an SVD algorithm on the dataset.
data = Dataset.load_from_df(user_data, reader)
algo_ = SVD(n_factors=20, n_epochs=10, reg_all=0.05)
algo_.fit(trainset)



In [71]:

# Then predict ratings for all pairs (u, i) that are NOT in the training set.

predictions = algo_.test(testset)





In [110]:
predictions

[Prediction(uid=733902, iid=162688, r_ui=4.0, est=4.53586869176104, details={'was_impossible': False}),
 Prediction(uid=61660, iid=243317, r_ui=5.0, est=4.800866824061569, details={'was_impossible': False}),
 Prediction(uid=131584, iid=23094, r_ui=5.0, est=4.572232014494212, details={'was_impossible': False}),
 Prediction(uid=74904, iid=24429, r_ui=5.0, est=4.268123846331667, details={'was_impossible': False}),
 Prediction(uid=847800, iid=7817, r_ui=5.0, est=4.313190478685142, details={'was_impossible': False}),
 Prediction(uid=592062, iid=62254, r_ui=5.0, est=4.093578631774795, details={'was_impossible': False}),
 Prediction(uid=83093, iid=193861, r_ui=5.0, est=4.974704078852973, details={'was_impossible': False}),
 Prediction(uid=235281, iid=151177, r_ui=5.0, est=4.70415251743551, details={'was_impossible': False}),
 Prediction(uid=655731, iid=349977, r_ui=5.0, est=4.342252837448786, details={'was_impossible': False}),
 Prediction(uid=2001525309, iid=231701, r_ui=5.0, est=4.441209192

In [82]:
recs = pd.DataFrame(predictions)

In [83]:
recs = recs.rename(columns={"uid": "user_id"})

In [104]:
filtered_preds = []
def diet_prefs(recs):
  for prediction in predictions:
    for user in user_diets:
      if user_diets['is_vegetarian'] == "True":
        filtered_preds = predictions[prediction-1].drop(prediction-1)
      elif user_diets['is_vegan'] == "True":
        filtered_preds = predictions[prediction-1].drop(prediction-1)
      elif user_diets['is_GF'] == "True":
        filtered_preds = predictions[prediction-1].drop(prediction-1)
      else:
        filtered_preds.append(prediction)


    #top_n_similar_users = user_similarities[user-1].drop(user-1)

In [105]:
recs.apply(diet_prefs)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [114]:
# Print the recommended items for each user
import collections
top_n = get_top_n(predictions, n=3)

for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

NameError: name 'defaultdict' is not defined

In [ ]:
user_data = user_data.drop(columns=['review', 'date'])

In [ ]:
for col in user_ratings:
    mean = user_ratings[col].mean()
    user_ratings[col] = user_ratings[col].fillna(value=mean)
user_ratings.head()

In [ ]:
import numpy as np
import datetime

In [ ]:
# ⏰ Expect this cell to take several minutes to run
start = datetime.datetime.now()
user_matrix = []
for i, row in enumerate(user_ratings.index):
    u1 = user_ratings[row]
    # Matrix is symetric, so fill in values for previously examined users
    user_distances = [entry[i] for entry in user_matrix]
    for j, row2 in enumerate(user_ratings.index[i:]):
        u2 = user_ratings[row2]
        d = distance(u1,u2)
        user_distances.append(d)
    user_matrix.append(user_distances)
user_similarities = pd.DataFrame(user_matrix)

end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

user_similarities.head()

NameError: name 'user_ratings' is not defined

In [ ]:
#function from rec. systems lesson, removes previously rated items and orders items for recommendations based on rating (high to low)
#based on avg of closest users ratings
def recommend_recipes(user, user_similarities, user_ratings, df, n_users=20, n_items=10):
    """n is the number of similar users who you wish to use to generate recommendations."""
    # User_Similarities Offset By 1 and Must Remove Current User
    top_n_similar_users = user_similarities[user-1].drop(user-1).sort_values().index[:n_users]
    # Again, fixing the offset of user_ids
    top_n_similar_users = [i+1 for i in top_n_similar_users]
    already_rated = set(df[df.user_id == 0].item_id.unique())
    unrated = set(df.item_id.unique()) - already_rated
    projected_user_ratings = user_ratings[user_ratings.index.isin(top_n_similar_users)].mean()[list(unrated)].sort_values(ascending=False)
    return projected_user_ratings[:n_items]

In [ ]:
recommend_movies(1, user_similarities, user_ratings, df)

In [ ]:
# return the top n recommendations using the
def recommended_recipes(user_ratings,movie_title_df,n):
        for idx, rec in enumerate(user_ratings):
            title = movie_title_df.loc[movie_title_df['movieId'] == int(rec[0])]['title']
            print('Recommendation #', idx+1, ': ', title, '/n')
            n-=1
            if n==0:
                break

recommended_movies(ranked_movies,df_movies,5)